In [21]:
# grid search lstm for monthly airline passengers dataset
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
from pandas import DataFrame
import pandas as pd
from keras import optimizers
import keras.backend as K
import numpy as np

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

def reset_weights(model):
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)

# transform list into supervised learning format
def series_to_supervised(data, n_in, n_out=1):
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	agg.dropna(inplace=True)
	return agg.values

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# root mean squared error or rmse
def measure_mae(actual, predicted):
	return mean_absolute_error(actual, predicted)

# difference dataset
def difference(data, order):
	return [data[i] - data[i - order] for i in range(order, len(data))]

# fit a model
def model_fit(train, test,config):
	# unpack config
	n_input, n_nodes, n_epochs, n_batch, n_diff = config
	#prepare data
	#print(test)      
	if n_diff > 0:
		train = difference(train, n_diff)
		test = difference(test, n_diff) 
	#print(test)          
	# transform series into supervised format 
	data = series_to_supervised(train[:-25], n_in=n_input)
	data_test = series_to_supervised(train[-24:], n_in=n_input)    
	# separate inputs and outputs
	train_x, train_y = data[:, :-1], data[:, -1]  
	test_x, test_y = data_test[:, :-1], data_test[:, -1]     
	# reshape input data into [samples, timesteps, features]
	n_features = 1
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
	test_x = test_x.reshape((test_x.shape[0], test_x.shape[1], n_features))    
	# define model
	model = Sequential()
	model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
	model.add(Dense(n_nodes, activation='relu'))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam', metrics=['mse','accuracy','mape'])
	plot_model(model, to_file='model_plot_lstm.png', show_shapes=True, show_layer_names=True)    
	# fit model
	initial_weights = model.get_weights()    
	# fit network
	repeats=1
	for i in range(repeats):
		model.set_weights(initial_weights)      
		model.compile(loss='mse', optimizer='adam', metrics=['mse','accuracy','mape'])          
		aa=model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch,validation_data=(test_x, test_y), verbose=0) 
		pyplot.plot(aa.history['loss'],  color='blue')
		pyplot.plot(aa.history['val_loss'], color='orange')
		#print('%d) TrainRMSE=%f, TestRMSE=%f' % (i, xx.history['loss'].iloc[-1], xx.history['val_loss'].iloc[-1]))
		model.reset_states() 
		#reset_weights(model)         
	pyplot.savefig('epochs_diagnostic_lstm.png')   
	pyplot.legend()
	pyplot.show()    
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model

# forecast with the fit model
def model_predict(model, history, config):
	# unpack config
	n_input, _, _, _, n_diff = config   
	# prepare data
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]      
		history = difference(history, n_diff)       
	# reshape sample into [samples, timesteps, features]
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# forecast  
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list() 
	# split dataset
	train, test = train_test_split(data, n_test)
	# fit model   
	model = model_fit(train,test, cfg)
	# seed history with training dataset
	history = [x for x in train]     
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = model_predict(model, history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])      
	# estimate prediction error
	error = measure_mae(test, predictions)
	pred=pd.DataFrame(predictions)   
	test_=pd.DataFrame(test)      
	pred=pd.concat([pred, test_], axis=1, ignore_index=True)
	pathx=r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\ONESTEP\LSTM\lstm_grid_predictions_MAE.xlsx'  
	export_pred = pred.to_excel (pathx,sheet_name='Tahmin_MAE', index = None, header=True)    
	print(' > %.3f' % error)
	return error

# score a model, return None on failure
def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
	# evaluate configs
	scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
	scores_=pd.DataFrame(scores)  
	path=r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\ONESTEP\LSTM\lstm_grid_scores_MAE_%s.xlsx' % n_test
	export_scores = scores_.to_excel (path,sheet='Hata_MAE', index = None, header=True)    
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a list of configs to try
def model_configs():
	# define scope of configs
	n_input = [12]
	n_nodes = [50,100]
	n_epochs = [50]
	n_batch = [1,50,100,200]
	n_diff = [1]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs

# define dataset
#series = read_csv('monthly-airline-passengers.csv', header=0, index_col=0)
#data = series.values
xxx = read_csv('MyData.csv', sep=',',  index_col=0, 
        usecols=[0,13], encoding='latin-1') 
yyy = xxx.values
#print(xxx)
# data split
n_test = 12
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(yyy, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)

Total configs: 8


ValueError: Initializer for variable lstm_26/kernel/ is from inside a control-flow construct, such as a loop or conditional. When creating a variable inside a loop or conditional, use a lambda as the initializer.

In [4]:
# grid search lstm for monthly airline passengers dataset
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model
from pandas import DataFrame
import pandas as pd

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# transform list into supervised learning format
def series_to_supervised(data, n_in, n_out=1):
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	agg.dropna(inplace=True)
	return agg.values

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# root mean squared error or rmse
def measure_mae(actual, predicted):
	return mean_absolute_error(actual, predicted)

# difference dataset
def difference(data, order):
	return [data[i] - data[i - order] for i in range(order, len(data))]

# fit a model
def model_fit(train, config):
	# unpack config
	n_input, n_nodes, n_epochs, n_batch, n_diff = config
	# prepare data
	if n_diff > 0:
		train = difference(train, n_diff)
	# transform series into supervised format   
	data = series_to_supervised(train, n_in=n_input)
	# separate inputs and outputs
	train_x, train_y = data[:, :-1], data[:, -1]  
	# reshape input data into [samples, timesteps, features]
	n_features = 1
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
	# define model
	model = Sequential()
	model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
	model.add(Dense(n_nodes, activation='relu'))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam')
	plot_model(model, to_file='model_plot_lstm.png', show_shapes=True, show_layer_names=True)    
	# fit model
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model

# forecast with the fit model
def model_predict(model, history, config):
	# unpack config
	n_input, _, _, _, n_diff = config    
	# prepare data
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]    
		history = difference(history, n_diff)       
	# reshape sample into [samples, timesteps, features]
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# forecast    
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# fit model   
	model = model_fit(train, cfg)
	# seed history with training dataset
	history = [x for x in train]    
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = model_predict(model, history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])       
	# estimate prediction error
	error = measure_rmse(test, predictions)
	pred=pd.DataFrame(predictions)   
	test_=pd.DataFrame(test)    
	pred=pd.concat([pred, test_], axis=1, ignore_index=True)
	export_pred = pred.to_excel (r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\chapter_15\lstm_grid_predictions_RMSE.xlsx', index = None, header=True)    
	print(' > %.3f' % error)
	return error

# score a model, return None on failure
def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
	# evaluate configs
	scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
	scores_=pd.DataFrame(scores)  
	path=r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\chapter_15\lstm_grid_scores_RMSE_%s.xlsx' % n_test
	writer = pd.ExcelWriter(path, engine='xlsxwriter') 
	print(scores)    
	print(scores[0][0])  
	export_scores = scores_.to_excel (writer,sheet_name='Hata_RMSE', index = None, header=True)      
	#export_scores = scores_.to_excel (path,sheet='deneme', index = None, header=True)    
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a list of configs to try
def model_configs():
	# define scope of configs
	n_input = [12]
	n_nodes = [100]
	n_epochs = [50]
	n_batch = [1]
	n_diff = [1,4]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs

# define dataset
#series = read_csv('monthly-airline-passengers.csv', header=0, index_col=0)
#data = series.values
xxx = read_csv('MyData.csv', sep=',',  index_col=0, 
        usecols=[0,13], encoding='latin-1') 
yyy = xxx.values
#print(xxx)
# data split
n_test = 12
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(yyy, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)

Total configs: 2
[[292.53416149]
 [272.17391304]
 [289.94409938]
 [272.10559006]
 [269.49068323]
 [165.8447205 ]
 [176.91304348]
 [193.38509317]
 [231.81987578]
 [274.99378882]
 [250.25892857]
 [254.36956522]]
length diff train: 
99
length train_x,train_y: 
87
87
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
 > 36.883
[[292.53416149]
 [272.17391304]
 [289.94409938]
 [272.10559006]
 [269.49068323]
 [165.8447205 ]
 [176.91304348]
 [193.38509317]
 [231.81987578]
 [274.99378882]
 [250.25892857]
 [254.36956522]]
length diff train: 
99
length train_x,train_y: 
87
87
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
12
x input
 > 40.776
[[292.53416149]
 [272.17391304]
 [289.94409938]
 [272.10559006]
 [269.49068323]
 [165.8447205 ]
 [176.91304348]
 [193.38509317]
 [231.81987578]
 [274.99378882]
 [250.25892857]
 [254.36956522]]
length diff 

NameError: name 'train' is not defined